In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import PIL

import torchvision as tv
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary

from pushover import notify
from utils import makegif
from random import randint

from IPython.display import Image
from IPython.core.display import Image, display

%load_ext autoreload
%autoreload 2

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
bs = 32 # batchsize

In [4]:
# Load Data

def load_img(img_path):
    img = PIL.Image.open(img_path)
#     img = PIL.ImageMath.eval("int(img)", img=img)
#     print(img)
#     img = PIL.ImageOps.grayscale(img)
    img = img.convert(mode="L")
#     print(img)
    return img
IMRANGE = 256 # uint8
dataset = datasets.ImageFolder(root='trainings/rolls_gray', transform=transforms.Compose([
#     transforms.Resize(64),
    transforms.ToTensor(), 
#     lambda x: (x * IMRANGE).type(torch.IntTensor),
#     lambda x: x.type(torch.FloatTensor),

]), loader=load_img)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=bs, shuffle=True)
len(dataset.imgs), len(dataloader)

# size of input = 3 x 128 x 128

(1339, 42)

In [5]:
# Fixed input for debugging
fixed_x, _ = next(iter(dataloader))
save_image(fixed_x, 'outputs/real_image.png')

# Image('outputs/real_image.png')

In [6]:
print(dataset[0][0].shape)
sample_dat = dataset[0][0]
# print(dataset[0])
# for i in range(128):
#     for j in range(128):
#         if sample_dat[0][i][j] > 0:
#             print(i, j, sample_dat[0][i][j])
HSIZE = 2048 #9216 # 1024
ZDIM =  32

torch.Size([1, 128, 128])


In [7]:
class Flatten(nn.Module):
    def forward(self, input):
#         print("flatten: ", input.shape)
        return input.view(input.size(0), -1)

In [8]:
class UnFlatten(nn.Module):
    def forward(self, input, size=HSIZE):
        return input.view(input.size(0), size, 1, 1)

In [9]:
class VAE(nn.Module):
    def __init__(self, image_channels=1, h_dim=HSIZE, z_dim=ZDIM):
        super(VAE, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, 32, kernel_size=4, stride=2), # -> [32, 32, 31, 31] 63
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2), # -> [32, 64, 14, 14] 31
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2), # -> [32, 128, 6, 6] 14
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2), # -> [32, 256, 2, 2] 6
            nn.ReLU(), 
            nn.Conv2d(256, 512, kernel_size=4, stride=2), # -> Null -> [32, 512, 2, 2] 
            nn.ReLU(), 
            Flatten() # -> [32, 1024]  -> [32, 2048]
            # [32, a, b, c] -> [32, abc]
        )
        
        self.fc1 = nn.Linear(h_dim, z_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(z_dim, h_dim)
        
        self.decoder = nn.Sequential(
            UnFlatten(), 
            nn.ConvTranspose2d(h_dim, 256 , kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
            nn.Sigmoid(),
        )
#         self.decoder = nn.Sequential(
#             UnFlatten(),
#             nn.ConvTranspose2d(h_dim, 128, kernel_size=5, stride=2),
#             nn.ReLU(),
#             nn.ConvTranspose2d(128, 64, kernel_size=5, stride=2),
#             nn.ReLU(),
#             nn.ConvTranspose2d(64, 32, kernel_size=6, stride=2),
#             nn.ReLU(),
#             nn.ConvTranspose2d(32, image_channels, kernel_size=6, stride=2),
#             nn.Sigmoid(),
#         )
        
    def reparameterize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        # return torch.normal(mu, std)
        esp = torch.randn(*mu.size())
        z = mu + std * esp
        return z
    
    def bottleneck(self, h):
        mu, logvar = self.fc1(h), self.fc2(h)
#         print("bottle: ",mu.shape, logvar.shape)
        z = self.reparameterize(mu, logvar)
        return z, mu, logvar

    def encode(self, x):
#         print("======== Encode ========", x.shape)
        h = self.encoder(x)
#         print("enc(x): ", h.shape)
        z, mu, logvar = self.bottleneck(h)
#         print("z.shape: ", z.shape)
        return z, mu, logvar

    def decode(self, z):
#         print("======== Decode ========", z.shape)
        z = self.fc3(z)
#         print("fc3(z).shape: ", z.shape)
        z = self.decoder(z)
#         print("decode(fc3(z)).shape: ", z.shape)
        return z

    def forward(self, x):
        z, mu, logvar = self.encode(x)
#         print(z.shape)
        z = self.decode(z)
#         print(z.shape, mu.shape, logvar.shape)
        return z, mu, logvar

In [10]:
image_channels = fixed_x.size(1)

In [11]:
vae = VAE(image_channels=image_channels).to(device)
# model.load_state_dict(torch.load('vae.torch', map_location='cpu'))

In [12]:
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

In [13]:
def loss_fn(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x, size_average=False)
#     BCE = F.mse_loss(recon_x, x, size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD, BCE, KLD

In [14]:
epochs = 50

In [15]:
for epoch in range(epochs):
    for idx, (images, _) in enumerate(dataloader):
        recon_images, mu, logvar = vae(images)
#         print(images.shape)
        loss, bce, kld = loss_fn(recon_images, images, mu, logvar)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#         to_print = "Epoch[{}/{}] Loss: {:.3f} {:.3f} {:.3f}".format(epoch+1, 
#                                 epochs, loss.data[0]/bs, bce.data[0]/bs, kld.data[0]/bs)
        to_print = "Epoch[{}/{}] Loss: {:.3f} {:.3f} {:.3f}".format(epoch+1, 
                                epochs, loss.data/bs, bce.data/bs, kld.data/bs)
    if epoch % 10 == 0 and epoch != 0:
        torch.save(vae.state_dict(), 'models/vae.torch-grayv-nimgs_{}-epochs_{}-{}'.format(len(dataset.imgs), epoch, epochs))
        
    print(to_print, loss.data, kld.data, bs)

# notify to android when finished training
notify(to_print, priority=1)

/usr/local/lib/python3.6/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Epoch[1/50] Loss: 3128.845 3124.562 4.283 tensor(100123.0547) tensor(137.0697) 32
Epoch[2/50] Loss: 318.860 318.489 0.370 tensor(10203.5068) tensor(11.8442) 32
Epoch[3/50] Loss: 296.285 295.983 0.302 tensor(9481.1357) tensor(9.6645) 32
Epoch[4/50] Loss: 331.195 330.912 0.283 tensor(10598.2295) tensor(9.0459) 32
Epoch[5/50] Loss: 322.957 322.722 0.236 tensor(10334.6357) tensor(7.5385) 32
Epoch[6/50] Loss: 297.832 297.580 0.252 tensor(9530.6123) tensor(8.0516) 32
Epoch[7/50] Loss: 325.332 325.095 0.237 tensor(10410.6104) tensor(7.5681) 32
Epoch[8/50] Loss: 307.353 307.099 0.255 tensor(9835.3037) tensor(8.1482) 32
Epoch[9/50] Loss: 300.357 300.127 0.231 tensor(9611.4395) tensor(7.3771) 32
Epoch[10/50] Loss: 337.542 337.310 0.232 tensor(10801.3438) tensor(7.4304) 32
Epoch[11/50] Loss: 326.421 326.212 0.209 tensor(10445.4805) tensor(6.6891) 32
Epoch[12/50] Loss: 296.958 296.715 0.243 tensor(9502.6475) tensor(7.7656) 32
Epoch[13/50] Loss: 307.886 307.672 0.214 tensor(9852.3457) tensor(6.8360

In [16]:
torch.save(vae.state_dict(), 'models/vae.torch-grayv-nimgs_{}-epochs_{}'.format(len(dataset.imgs), epochs))